In [1]:
import os
import re

import pandas as pd

from orv_cleanup_utils import clean_up

pd.set_option('display.max_columns', None)

In [2]:
path = '/home/common/regulation_data/parsed_reports/'

In [3]:
# main_df – таблица, куда сложены заполнения one-to-one
main_df = pd.read_csv(path + 'main_df.csv')

In [4]:
# загрузка one-to-many таблиц
otm_tables_fnames = sorted([fn for fn in os.listdir(path) if not (fn.startswith('main') or fn.startswith('.'))])
otm_tables = {fn[:-4]: pd.read_csv(path + fn) for fn in otm_tables_fnames}
goals_df = otm_tables['goals']
groups_df = otm_tables['groups']
expenses_df = otm_tables['expenses']
group_changes_df = otm_tables['group_changes']
group_expenses_df = otm_tables['group_expenses']
risks_df = otm_tables['risks']
necessary_measures_df = otm_tables['neccessary_measures']
cancel_duties_df  = otm_tables['cancel_duties']
new_functions_df = otm_tables['new_functions']

## Забираем мусорные заполнения из гуглодиска

In [5]:
import gspread

from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials \
        .from_json_keyfile_name('cpur-project.json', scope)
client = gspread.authorize(credentials)

sheet = client.open('ОРВ: мусорные заполнения')

In [6]:
import json 

# Забираем маппинг названий в датафрейме и в экселевском файле
with open('junk/mapping_fields.json') as fp:
    all_fields = json.load(fp)

In [7]:
import time
import os

update = False

# Получаем значения мусорных заполнений по экспертным оценкам
# из Гугл.Диска и сохраняем локально
if os.path.exists('junk/junk_by_field.json') and update is False:
    with open('junk/junk_by_field.json', 'r') as fp:
        junk_by_field = json.load(fp)
else:
    junk_by_field = {}
    for table in all_fields.values():
        for sheet_name in table.keys():
            junk = set()
            for _ in sheet.worksheet(sheet_name + '_freq').get_all_values():
                if _[2].strip() == '0':
                    junk.add(_[0])
            for _ in sheet.worksheet(sheet_name + '_short').get_all_values():
                if _[2].strip() == '0':
                    junk.add(_[0])
            junk_by_field[sheet_name] = list(junk)
            time.sleep(3)
    with open('junk/junk_by_field.json', 'w') as fp:
        json.dump(junk_by_field, fp)

## Функция обработки датафрейма

In [8]:
def fill_df_info(dataframe, df_name):
    """
    Функция принимает датафрейм, сравнивает значение нужных полей
    с мусорными, проставляет соответствующую оценку
    и возвращает новый датафрейм
    """
    if df_name not in all_fields.keys():
        print('No such table in dataset.')
        return
    
    if 'header: id' in dataframe.columns:
        id_column = 'header: id'
    else:
        id_column = 'id'
    
    result_df = pd.DataFrame()
    for index, row in dataframe.iterrows():
        result_df.loc[index, id_column] = row[id_column]
        for short, long in all_fields.get(df_name).items():    
            value = dataframe.loc[index, long]
            value = clean_up(value)

            # Проверяем, есть ли текущее вхождение 
            # в "мусоре" для этого поляgoals@timing
            junk_set = junk_by_field.get(short)
            if junk_set is not None and value in junk_set:
                valid = 0
            else:        
                valid = 1

            result_df.loc[index, short] = value
            result_df.loc[index, str(short + '_valid')] = valid
    return result_df

### Получаем обработанный основной датафрейм

In [9]:
junk_main_df = fill_df_info(main_df, 'main')

### Получаем вспомогательные датафреймы

In [10]:
for _ in otm_tables.keys():
    if _ in all_fields.keys():
        df = otm_tables.get(_)
        result = fill_df_info(df, _)
#         result.to_excel(f'junk_dfs/junk_{_}_df.xlsx', header=True)

## Получаем информационные поля из базы 

In [11]:
from datetime import date
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    host="130.193.43.155",
    database="orv_reports",
    user="reports_master",
    password="master_reports")

In [12]:
c = conn.cursor()
statement = "SELECT grid.projectid, grid.date, grid.department, grid.okved, grid.regulatoryimpact FROM grid WHERE grid.procedure='Оценка регулирующего воздействия'"
c.execute(statement)
orv_info = c.fetchall()
c.close()

orv_info = pd.DataFrame(orv_info, columns=['projectid', 'date', 'department', 'okved', 'impact'])

In [13]:
def date_to_year(date):
    matches = re.findall('\d{4}', str(date))
    if len(matches) > 0:
        return matches[0]
    else:
        return ''

In [14]:
orv_info['date'] = orv_info['date'].apply(date_to_year)
orv_info.head(1)

,projectid,date,department,okved,impact
0,02/04/06-21/00116602,2021,Минэкономразвития России,Корпоративное управление,Средняя


### Присоединяем поля из базы к мусорным заполнениям

In [15]:
junk_main_df = orv_info.merge(junk_main_df, right_on='header: id', left_on='projectid')
junk_main_df = junk_main_df.drop(columns=['header: id'])

In [ ]:
# junk_main_df.to_excel('junk/orv_info.xlsx', header=True)